# LangChain 学习 - Prompts & Messages

## 本节目标
- 理解Prompt Template的作用
- 掌握ChatPromptTemplate的使用
- 学会变量注入和动态提示词
- **理解Pipe操作符（|）和链式使用**
- 掌握Few-shot Learning技术
- 学习提示词工程最佳实践

## 为什么需要Prompt Templates？
在实际应用中，我们经常需要：
1. **复用**：同一个提示词框架，不同的输入内容
2. **维护**：集中管理提示词，方便修改和优化
3. **组合**：将多个提示词组合成复杂的对话流程

Prompt Template让提示词变成可编程的组件！

In [1]:
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI

# 初始化模型
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=config.OPENAI_API_KEY,
    base_url=config.OPENAI_BASE_URL,
)

print("环境准备完成")

环境准备完成


## 问题：硬编码的提示词
在第一课中，我们这样使用提示词：

```python
response = model.invoke("给我讲一个关于AI的笑话")
```

**问题**：
- 每次都要写完整的提示词
- 难以复用和维护
- 无法动态插入变量
- 团队协作困难

**解决方案**：Prompt Templates!

In [2]:
from langchain_core.prompts import ChatPromptTemplate

# 创建一个简单的提示词模板
prompt = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话")

print("模板创建成功！")
print(f"模板内容：{prompt}")
print(f"\n变量：{prompt.input_variables}")

模板创建成功！
模板内容：input_variables=['topic'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='给我讲一个关于{topic}的笑话'), additional_kwargs={})]

变量：['topic']


In [3]:
# 方式1：format() - 生成字符串
formatted_prompt = prompt.format(topic="程序员")
print("格式化后的提示词：")
print(formatted_prompt)

print("\n" + "="*60 + "\n")

# 方式2：format_messages(topic="AI")
messages = prompt.format_messages(topic="AI")
print("格式化后的消息对象：")
for msg in messages:
    print(f" {type(msg).__name__}: {msg.content}")

格式化后的提示词：
Human: 给我讲一个关于程序员的笑话


格式化后的消息对象：
 HumanMessage: 给我讲一个关于AI的笑话


## ⭐ Pipe操作符（|）- 链式调用
这是LangChain最核心的语法之一！

### 概念
Pipe操作符（`|`）用于连接不同的组件，形成处理链（Chain）

### 语法
```python
chain = prompt | model
result = chain.invoke({"topic": "Python"})
```

### 优势
- 代码简洁易读
- 可组合性强
- 这是LCEL（LangChain Expression Language）的基础

In [4]:
# 创建一个chain：prompt | model
chain = prompt | model

# 调用chain
response = chain.invoke({"topic": "Python"})

print("AI回复：")
print(response.content)

AI回复：
当然可以！这是一个关于Python的笑话：

为什么Python程序员喜欢用“while True”？

因为他们总是想保持“循环”快乐！😄

希望你喜欢这个笑话！如果想听更多，随时告诉我！


In [5]:
# Chain也支持stream
print("=" * 60)
print("使用Chain进行流式输出改造")
print("=" * 60)

chain = prompt | model

print("AI回复（流式）：", end="", flush=True)
for chunk in chain.stream({"topic": "猫"}):
    print(chunk.content, end="", flush=True)

print("\n")

使用Chain进行流式输出改造
AI回复（流式）：当然可以！这里有一个关于猫的笑话：

为什么猫喜欢在电脑键盘上走来走去？

因为它们想要找到“鼠标”！🐭😸

希望你喜欢这个笑话！



## ChatPromptTemplate - 多消息模板

`from_template()`只能创建单挑消息，实际应用中我们需要多条消息（system + user）

使用`from_messages()`创建包含多条消息的模板：

```python
ChatPromptTemplate.from_messages([
    ("system", "系统消息"),
    ("user", "用户消息"),
    ("assistant", "AI消息"),  # 可选
])
```

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# 创建包含system和user消息的模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是{role}，请用专业的方式回答问题。"),
    ("user", "{question}")
])

print("模板变量：", prompt.input_variables)
print("\n格式化后的消息：")

messages = prompt.format_messages(
    role="Python编程专家",
    question="什么是装饰器？"
)

for msg in messages:
    print(f" {msg.__class__.__name__}: {msg.content}")

模板变量： ['question', 'role']

格式化后的消息：
 SystemMessage: 你是Python编程专家，请用专业的方式回答问题。
 HumanMessage: 什么是装饰器？


In [7]:
# 创建一个角色扮演助手模板
role_play_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位{role}，说话风格是{style}。"),
    ("user", "{question}")
])

# 创建chain
chain = role_play_prompt | model

# 测试不同角色
print("【示例1：海盗船长】")
response1 = chain.invoke({
    "role": "海盗船长",
    "style": "粗犷豪迈，常用'哈哈'和'伙计们'",
    "question": "今天天气怎么样？"
})
print(response1.content)

print("\n" + "="*60 + "\n")

print("【示例2：文艺诗人】")
response2 = chain.invoke({
    "role": "文艺诗人",
    "style": "优雅浪漫，善用比喻",
    "question": "今天天气怎么样？"
})
print(response2.content)

【示例1：海盗船长】
哈哈，伙计们！今天天气可真不错！海风徐徐，阳光普照，正是航海的好时光！咱们可以扬帆起航，去征服那无尽的海洋！准备好你们的帆和水手刀，出发吧！


【示例2：文艺诗人】
今天的天气，如同一幅细腻的画卷，阳光洒落在大地上，宛如金色的丝线编织而成的梦；微风轻拂，似那温柔的低语，带着春天的气息，轻轻吻过每一片绿叶。天空湛蓝，云朵如同棉花糖般悠然漂浮，仿佛在诉说着无尽的故事。无论是漫步于大街小巷，还是静坐于窗前，都是与自然亲密接触的绝佳时刻。这样的日子，仿佛是上帝为我们准备的一场盛宴，让心灵在温暖的阳光下尽情舞动。


## 多变量模板

模板可以包含多个变量，让提示词更加灵活：

```python
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是{role}，擅长{skill}。"),
    ("user", "关于{topic}: {question}")
])
```

**调用时必须提供所有变量**：
```python
chain.invoke({
    "role": "...",
    "skill": "...",
    "topic": "...",
    "question": "...",
})
```

调用时必须提供所有变量：

```python
chain.invoke({
    "role": "...",
    "skill": "...",
    "topic": "...",
    "question": "...",
})


chain = complex_prompt | model


response = chain.invoke({
    "role": "技术博主",
    "specialty": "Python和AI",
    "style": "通俗易懂，多用比喻",
    "length": "100字左右",
    "question": "什么是LangChain？"
})

print(response.content)
```

In [8]:
# 创建复杂的多变量模板
complex_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一位{role}，专长于{specialty}。
你的回答风格：{style}
回答长度：{length}"""),
    ("user", "{question}")
])

chain = complex_prompt | model

response = chain.invoke({
    "role": "技术博主",
    "specialty": "Python和AI",
    "style": "通俗易懂，多用比喻",
    "length": "100字左右",
    "question": "什么是LangChain？"
})

print(response.content)

LangChain就像是一个多功能工具箱，专门为构建与语言模型互动的应用而设计。想象一下，你在使用一个强大的机器人助手，LangChain帮助你更好地组织和利用这个助手的能力。它提供了连接不同数据源、处理文本、生成响应等功能，让开发者可以轻松创建聊天机器人、问答系统等应用。通过这个工具箱，开发者可以有效地“搭建”与语言模型的桥梁，使得复杂的对话变得简单易行。


## Few-shot Learning（少样本学习）

### 概念
通过提供**示例**来引导模型理解任务，而不是详细描述规则。

### 为什么重要？
System：任务说明
Human：示例输入1
AI：示例输出1
Human：示例输入2
AI：示例输出2
Human：实际输入

from langchain_core.prompts import ChatPromptTemplate

# 创建few-shot prompt用于情感分类
sentiment_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个情感分类器，将文本分类为：积极、消极、中性"),

    # Few-shot examples
    ("human", "这个产品太棒了！非常满意！"),
    ("ai", "积极"),

    ("human", "质量很差，完全不值这个价格。"),
    ("ai", "消极"),

    ("human", "包装还可以，物流正常。"),
    ("ai", "中性"),
    
    # 实际任务
    ("human", "{text}")
])

chain = sentiment_prompt | model

# 测试
```python
test_texts = [
    "这家餐厅的服务非常好，菜品也很美味！",
    "等了一个小时才上菜，太慢了。",
    "价格适中，环境一般。"
]

for text in test_texts:
    result = chain.invoke({"text": text})
    print(f"输入：{text}")
    print(f"分类：{result.content}\n")
```

In [9]:
from langchain_core.prompts import ChatPromptTemplate

# 创建few-shot prompt用于情感分类
sentiment_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个情感分类器，将文本分类为：积极、消极、中性"),
    
    # Few-shot examples
    ("human", "这个产品太棒了！非常满意！"),
    ("ai", "积极"),
    
    ("human", "质量很差，完全不值这个价格。"),
    ("ai", "消极"),
    
    ("human", "包装还可以，物流正常。"),
    ("ai", "中性"),
    
    # 实际任务
    ("human", "{text}")
])

chain = sentiment_prompt | model

# 测试
test_texts = [
    "这家餐厅的服务非常好，菜品也很美味！",
    "等了一个小时才上菜，太慢了。",
    "价格适中，环境一般。"
]

for text in test_texts:
    result = chain.invoke({"text": text})
    print(f"输入：{text}")
    print(f"分类：{result.content}\n")

输入：这家餐厅的服务非常好，菜品也很美味！
分类：积极

输入：等了一个小时才上菜，太慢了。
分类：消极

输入：价格适中，环境一般。
分类：中性



In [10]:
# 创建few-shot prompt用于结构化输出（修复：转义JSON中的{}）
extract_prompt = ChatPromptTemplate.from_messages([
    ("system", "从文本中提取人名、地点、时间，以JSON格式输出"),
    
    # Few-shot examples - 用{{}}转义JSON的{}
    ("human", "张三昨天在北京参加了会议"),
    ("ai", '{{"人名": "张三", "地点": "北京", "时间": "昨天"}}'),
    
    ("human", "李四下周要去上海出差"),
    ("ai", '{{"人名": "李四", "地点": "上海", "时间": "下周"}}'),
    
    # 实际任务
    ("human", "{text}")
])

chain = extract_prompt | model

result = chain.invoke({"text": "王五今天早上在杭州开了个重要会议"})
print(result.content)

{"人名": "王五", "地点": "杭州", "时间": "今天早上"}


## Partial Variables（部分变量）

有时我们希望**预先填充**某些变量，只在调用时填充其他变量。

### 使用场景
- 固定的系统角色
- 当前日期/时间
- 配置信息

In [11]:
# 方法1：创建时指定
prompt = prompt.partial(role="Python专家")

# 方法2：动态函数
from datetime import datetime
prompt = prompt.partial(date=lambda: datetime.now().strftime("%Y-%m-%d"))
print(prompt)

input_variables=['question'] input_types={} partial_variables={'role': 'Python专家', 'date': <function <lambda> at 0x7f21740793f0>} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['role'], input_types={}, partial_variables={}, template='你是{role}，请用专业的方式回答问题。'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]


In [13]:
from datetime import datetime

# 创建模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是{role}，今天是{date}"),
    ("user", "{question}"),
])

# 使用partial预填充某些变量
partial_prompt = prompt.partial(
    role="AI助手",
    date=lambda: datetime.now().strftime("%Y年%m月%d日")
)

# 现在只需要提供question
chain = partial_prompt | model

response = chain.invoke({"question": "今天星期几？"})
print(response.content)

今天是2025年12月16日，星期二。


## 提示词工程最佳实践
### 1. **明确角色和任务**
```python
"你是一位Python编程专家，擅长解释复杂概念。"
```

### 2. **提供上下文**
```python
"在企业应用开发的场景下，回答以下问题..."
````

### 3. **使用Few-shot Examples**
- 提供2-5个高质量示例
- 示例要覆盖不同情况

### 4. ** 指定输出格式**
```python
"请以JSON格式输出，包含title和content两个字段"
```

### 5. **分步骤思考（Chain of Thought）**
```python
"让我们一步一步思考：\n1. 首先...\n2. 然后...\n3. 最后..."
```

### 6. **设置约束条件**
```python
"回答不超过100字"
"只使用Python标准库"
```

# 综合应用最佳实践
```python
best_practice_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一位经验丰富的代码审查员，专注于Python代码质量。

审查标准：
1. 代码风格（PEP 8）
2. 性能问题
3. 潜在bug
4. 安全隐患

输出格式：
- 问题等级：高/中/低
- 问题描述
- 修改建议

示例：
代码：x=[ i for i  in range(100) if i%2==0]
问题：
- 等级：低
- 描述：代码格式不规范，缺少空格
- 建议：x = [i for i in range(100) if i % 2 == 0]
"""),
    ("user", "请审查以下代码：\n{code}")
])

chain = best_practice_prompt | model

code_to_review = """
def calc(a,b):
    return a/b
"""

response = chain.invoke({"code": code_to_review})
print(response.content)
```

In [14]:
# 实战：创建一个支持多种风格的翻译器
translator_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一位专业翻译，将{source_lang}翻译成{target_lang}。

    翻译风格：{style}

    注意事项：
    - 保持原文的语气和情感
    - 参考文化差异
    - 确保准确性"""),

    # Few-shot examples
    ("human", "Hello, how are you?"),
    ("ai", "你好，你好吗？"),

    ("human", "{text}")
])

chain = translator_prompt | model

# 测试不同风格
styles = ["正式", "口语化", "文学性"]

text = "I can't believe this happened!"

for style in styles:
    print(f"\n【{style}风格】")
    response = chain.invoke({
        "source_lang": "英语",
        "target_lang": "中文",
        "style": style,
        "text": text
    })

    print(response.content)


【正式风格】
我简直不敢相信这件事发生了！

【口语化风格】
我真不敢相信会发生这样的事！

【文学性风格】
我简直不敢相信这居然发生了！


## 本节小结

✅ 你已经学会：
1. **Prompt Templates的作用和创建**
   - `from_template()` - 简单模板
   - `from_messages()` - 多消息模板
2. **⭐ Pipe操作符（|）** - 链式调用的核心
   - `prompt | model` 形成处理链
   - 支持invoke()和stream()
3. **变量注入和动态提示词**
   - 单变量和多变量
   - Partial variables预填充
4. **Few-shot Learning**
   - 通过示例引导模型
   - 提高输出质量和一致性
5. **提示词工程最佳实践**
   - 明确角色和任务
   - 提供上下文
   - 指定格式
   - 设置约束

🎯 核心概念：
- **模板化** = 可复用 + 可维护
- **Pipe操作符** = LangChain的灵魂
- **Few-shot** = 展示优于描述

💡 最佳实践：
- 将常用的提示词做成模板
- 使用Few-shot提高稳定性
- Partial填充固定变量
- 用Pipe组合多个步骤